In [1]:
!pip install BayesianOptimization

In [2]:
!pip install pycaret

     |████████████████████████████████| 256kB 7.7MB/s 
     |████████████████████████████████| 276kB 13.0MB/s 
     |████████████████████████████████| 157.5MB 81kB/s 
     |████████████████████████████████| 1.6MB 48.2MB/s 
     |████████████████████████████████| 14.2MB 328kB/s 
     |████████████████████████████████| 6.8MB 37.6MB/s 
     |████████████████████████████████| 245kB 42.0MB/s 
     |████████████████████████████████| 65.8MB 59kB/s 
     |████████████████████████████████| 174kB 42.0MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 1.8MB 35.2MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 1.1MB 38.5MB/s 
     |████████████████████████████████| 163kB 44.0MB/s 
     |████████████████████████████████| 153kB 45.3MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 348kB 48.4MB/s 
     |████████████████████████████████| 604kB 48.6MB/s

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc #garbage collection 감시
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
import seaborn as sns
import joblib
from functools import partial
import BayesianOptimization
from lightgbm import LGBMClassifier
import pycaret.classification


warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    min  = int(x[10:12])
    sec  = int(x[12:])
    return dt.datetime(year, month, day, hour,min,sec)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)
    
def string2num2(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    return int(x)

In [4]:
train_err_chg_prob = pd.read_csv("Jupyter Temp/lg 공모전/model sets/train_xy.csv")
test_err_chg = pd.read_csv("Jupyter Temp/lg 공모전/model sets/test_x.csv")

FileNotFoundError: ignored

In [ ]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y
sample_submission = pd.read_csv("Jupyter Temp/lg 공모전/sample_submission.csv")

train_x = train_err_chg_prob.drop(["user_id"],axis=1).copy()
train_y = train_err_chg_prob.copy()["label"]

test_x = test_err_chg.drop(["user_id"],axis=1).copy()

print(train_x.shape)
print(train_y.shape)

print(test_x.shape)
print(sample_submission.shape)

del train_err_chg_prob
del test_err_chg

(15000, 56)
(15000,)
(14998, 55)
(14999, 2)


In [ ]:
display(train_x.dtypes)
display(train_y.dtype)
display(test_x.dtypes)

quality_0       float64
quality_1       float64
quality_2       float64
quality_5       float64
quality_6       float64
quality_7       float64
quality_10      float64
quality_11      float64
quality_0_2     float64
quality_1_8     float64
quality_0_9     float64
quality_2_9     float64
quality_10_5    float64
quality_6_7     float64
label             int64
errtype_1         int64
errtype_2       float64
errtype_3       float64
errtype_4       float64
errtype_5       float64
errtype_6       float64
errtype_7       float64
errtype_8         int64
errtype_9         int64
errtype_10      float64
errtype_11      float64
errtype_12      float64
errtype_13      float64
errtype_14      float64
errtype_15      float64
errtype_16      float64
errtype_17      float64
errtype_18        int64
errtype_19        int64
errtype_20        int64
errtype_21        int64
errtype_22      float64
errtype_23      float64
errtype_24      float64
errtype_25        int64
errtype_26        int64
errtype_27      

dtype('int64')

quality_0       float64
quality_1       float64
quality_2       float64
quality_5       float64
quality_6       float64
quality_7       float64
quality_10      float64
quality_11      float64
quality_0_2     float64
quality_1_8     float64
quality_0_9     float64
quality_2_9     float64
quality_10_5    float64
quality_6_7     float64
errtype_1         int64
errtype_2       float64
errtype_3       float64
errtype_4       float64
errtype_5       float64
errtype_6       float64
errtype_7       float64
errtype_8         int64
errtype_9         int64
errtype_10      float64
errtype_11      float64
errtype_12      float64
errtype_13      float64
errtype_14      float64
errtype_15      float64
errtype_16      float64
errtype_17      float64
errtype_18        int64
errtype_19        int64
errtype_20        int64
errtype_21        int64
errtype_22      float64
errtype_23      float64
errtype_24      float64
errtype_25        int64
errtype_26      float64
errtype_27      float64
errtype_28      

In [ ]:
train_y = train_y.apply(lambda x: int(x))

In [ ]:
train_x.reset_index(drop=True,inplace=True)
test_x.reset_index(drop=True,inplace=True)

In [ ]:
train_x

,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_10,quality_11,quality_0_2,quality_1_8,quality_0_9,quality_2_9,quality_10_5,quality_6_7,label,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,0.0,0.0,0.0,1304.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1304.0,0.0,0,0,0.0,8.0,104.0,0.0,1.0,1.0,0,0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0,0,0,0,0.0,0.0,0.0,0,32,1.0,1.0,0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0.0,53.0,1.0,1.0,0,0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4,1,3,1,756.0,751.0,5.0,1,22,0.0,0.0,0,250.0,0.0,10,18.0,0.0,1,1,0,0.0,113.0,56.0,1
2,1200.0,14.0,670.0,863.0,2479.0,0.0,0.0,14.0,530.0,14.0,1200.0,670.0,863.0,2479.0,0,0,0.0,2.0,132.0,1.0,2.0,1.0,0,0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0,0,0,0,0.0,0.0,2.0,0,25,2.0,2.0,0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,2.0,1.0,1.0,0,0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0,0,0,0,28.0,19.0,0.0,0,59,0.0,0.0,0,65.0,0.0,8,0.0,0.0,1,1,2,0.0,17.0,1.0,0
4,570.0,57.0,570.0,702.0,51213.0,0.0,0.0,57.0,0.0,57.0,570.0,570.0,702.0,51213.0,1,0,0.0,0.0,1.0,0.0,3.0,4.0,0,0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0,0,0,0,140.0,119.0,0.0,0,33,0.0,0.0,0,176.0,0.0,16,0.0,0.0,1,1,0,0.0,4.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1940.0,194.0,1940.0,427.0,194.0,0.0,0.0,194.0,0.0,194.0,1940.0,1940.0,427.0,194.0,0,0,0.0,0.0,0.0,2.0,5.0,5.0,0,0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0,0,0,0,17.0,12.0,0.0,0,22,0.0,0.0,0,31.0,0.0,5,0.0,0.0,0,0,0,0.0,9.0,7.0,4
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,1,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0
14997,4650.0,465.0,4650.0,4179.0,465.0,0.0,0.0,465.0,0.0,465.0,4650.0,4650.0,4179.0,465.0,1,0,0.0,0.0,1.0,8.0,1.0,1.0,0,0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0,0,0,0,50.0,29.0,0.0,0,49,0.0,0.0,0,223.0,0.0,16,17.0,0.0,1,1,0,0.0,58.0,8.0,5
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0,0,0,0,7.0,1.0,0.0,0,4,0.0,0.0,0,14.0,0.0,15,12.0,0.0,1,1,0,0.0,6.0,0.0,0


In [ ]:
test_x

,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_10,quality_11,quality_0_2,quality_1_8,quality_0_9,quality_2_9,quality_10_5,quality_6_7,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,0.0,0.0,0.0,126.0,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.0,315.0,0,0.0,0.0,0.0,62.0,1.0,1.0,0,0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3,0,1,0,983.0,1308.0,5.0,2,32.0,0.0,0.0,0,83.0,2.0,16.0,0.0,1.0,3,3,0,2.0,92.0,0.0,2
1,330.0,33.0,330.0,248606.0,33.0,0.0,0.0,33.0,0.0,33.0,330.0,330.0,248606.0,33.0,0,0.0,3.0,98.0,3.0,4.0,4.0,0,0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0,0,0,0,0.0,0.0,0.0,0,24.0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0
2,4460.0,446.0,4460.0,3500.0,446.0,0.0,0.0,446.0,0.0,446.0,4460.0,4460.0,3500.0,446.0,0,0.0,0.0,0.0,79.0,2.0,1.0,0,0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0,0,0,0,60.0,40.0,2.0,0,40.0,0.0,0.0,0,230.0,0.0,16.0,0.0,0.0,1,1,0,14.0,113.0,1.0,2
3,0.0,0.0,0.0,642.0,13053.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,642.0,13053.0,0,0.0,0.0,0.0,107.0,2.0,2.0,0,0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1,0,1,0,3.0,2.0,0.0,0,5.0,2.0,2.0,0,134.0,0.0,4.0,16.0,3.0,1,1,0,0.0,40.0,8.0,0
4,2550.0,255.0,2550.0,1501.0,11727.0,0.0,0.0,255.0,0.0,255.0,2550.0,2550.0,1501.0,11727.0,0,0.0,0.0,2.0,6.0,1.0,1.0,0,0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0,0,0,0,10.0,8.0,0.0,0,14.0,1.0,1.0,0,332.0,152.0,11.0,6.0,1.0,1,1,1,0.0,48.0,90.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,11150.0,1115.0,11150.0,1115.0,1115.0,0.0,0.0,1115.0,0.0,1115.0,11150.0,11150.0,1115.0,1115.0,0,0.0,0.0,2.0,12.0,5.0,5.0,0,0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0,0,0,0,197.0,204.0,0.0,0,31.0,0.0,0.0,0,179.0,2.0,16.0,0.0,0.0,0,0,0,0.0,100.0,87.0,0
14994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,2.0,2.0,0,0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0,183.0,0.0,14.0,1.0,0.0,1,1,0,0.0,12.0,17.0,0
14995,2150.0,215.0,2150.0,8681.0,215.0,0.0,0.0,215.0,0.0,215.0,2150.0,2150.0,8681.0,215.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0,0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0,0,0,0,138.0,165.0,0.0,0,48.0,0.0,0.0,0,163.0,0.0,12.0,1118.0,0.0,1,1,1,0.0,329.0,2.0,3
14996,0.0,0.0,0.0,51227.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51227.0,0.0,0,0.0,0.0,1.0,4.0,2.0,2.0,0,0,0.0,15.0,16.0,0.0,1.0,73.0,44.0,0.0,1,0,1,0,11127.0,13161.0,0.0,3,44.0,0.0,0.0,0,78.0,0.0,14.0,0.0,0.0,1,1,3,0.0,30.0,49.0,0


In [ ]:
train_y = train_y.reset_index

In [ ]:
var_list = list(train_x.columns)
var_list.remove("label")

In [6]:
train = pd.read_csv("/content/drive/MyDrive/YBIGTA/21-1 겨울방학/Jupyter Temp/lg 공모전/0126 data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/YBIGTA/21-1 겨울방학/Jupyter Temp/lg 공모전/0126 data/test.csv")

In [8]:
train.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_3,q_meanmax_4,q_meanmax_5,q_meanmax_6,q_meanmax_7,q_meanmax_8,q_meanmax_9,q_meanmax_10,q_meanmax_11,q_meanmax_12,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_11,label,weekend_04,weekend_28,weekend_29,weekend_35
0,10000,1,0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,8.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,120.0,28.0,0.0,0.0
1,10001,1,1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.967949,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,10002,1,0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,1.666667,0.166667,1.666667,0.0,0.0,0.666667,3.666667,44.0,0.0,1.0,8.0,0.166667,0.0,2.0,1.0,2.0,0.0,0.0,12.0,3.0,1.0,0.0,216.0,0.0,0.0,0.0
3,10003,1,1,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.947059,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,1,1,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.833977,1.666667,0.166667,1.666667,0.0,0.0,0.416667,7.250000,87.0,0.0,0.0,3.0,0.166667,0.0,1.0,1.0,1.0,0.0,0.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
list(train.columns).startswith("q")

In [ ]:
train_q = train[]

In [7]:
clf = setup(data = train_x, target = 'label',numeric_features=var_list) 

NameError: ignored

In [ ]:
best_5 = compare_models(sort = 'Accuracy', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7863,0.8151,0.4937,0.7717,0.6017,0.4651,0.4871,11.0320
lightgbm,Light Gradient Boosting Machine,0.7845,0.8103,0.5033,0.7570,0.6042,0.4643,0.4828,0.2940
gbc,Gradient Boosting Classifier,0.7832,0.8126,0.4652,0.7859,0.5840,0.4501,0.4787,1.6650
et,Extra Trees Classifier,0.7813,0.8063,0.4766,0.7679,0.5877,0.4498,0.4738,1.0970
rf,Random Forest Classifier,0.7792,0.8077,0.4655,0.7694,0.5797,0.4420,0.4679,1.0170
ada,Ada Boost Classifier,0.7757,0.7978,0.4687,0.7525,0.5773,0.4358,0.4585,0.4750
xgboost,Extreme Gradient Boosting,0.7725,0.7975,0.5179,0.7092,0.5982,0.4450,0.4560,1.6090
lda,Linear Discriminant Analysis,0.7552,0.7492,0.3371,0.7993,0.4737,0.3470,0.4022,0.0790
ridge,Ridge Classifier,0.7494,0.0000,0.3185,0.7906,0.4534,0.3274,0.3846,0.0360
qda,Quadratic Discriminant Analysis,0.7264,0.7475,0.2413,0.7596,0.3612,0.2451,0.3115,0.0530


In [ ]:
best_5

 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                random_state=4898, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                            n_iter_no_change=None, presort='deprecated',
           

In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7929,0.8278,0.5051,0.7851,0.6147,0.4820,0.5039
1,0.7829,0.8058,0.4702,0.7783,0.5862,0.4509,0.4772
2,0.7848,0.8188,0.4876,0.7701,0.5971,0.4602,0.4826
3,0.7886,0.8183,0.4760,0.7956,0.5956,0.4645,0.4926
4,0.7799,0.8082,0.4891,0.7517,0.5926,0.4509,0.4705
Mean,0.7858,0.8158,0.4856,0.7762,0.5973,0.4617,0.4854
SD,0.0045,0.0080,0.0121,0.0148,0.0095,0.0115,0.0118


In [ ]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7738,0.7986,0.4598,0.7639,0.5741,0.4328,0.4586


In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test_x)

In [ ]:
predictions

,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_10,quality_11,quality_0_2,quality_1_8,...,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,Label,Score
0,0.0,0.0,0.0,126.0,315.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3,3,0,2.0,92.0,0.0,2,1,0.9508
1,330.0,33.0,330.0,248606.0,33.0,0.0,0.0,33.0,0.0,33.0,...,0.0,0,0,0,0.0,0.0,0.0,0,0,0.7573
2,4460.0,446.0,4460.0,3500.0,446.0,0.0,0.0,446.0,0.0,446.0,...,0.0,1,1,0,14.0,113.0,1.0,2,0,0.7551
3,0.0,0.0,0.0,642.0,13053.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1,1,0,0.0,40.0,8.0,0,1,0.8062
4,2550.0,255.0,2550.0,1501.0,11727.0,0.0,0.0,255.0,0.0,255.0,...,1.0,1,1,1,0.0,48.0,90.0,0,1,0.7251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,11150.0,1115.0,11150.0,1115.0,1115.0,0.0,0.0,1115.0,0.0,1115.0,...,0.0,0,0,0,0.0,100.0,87.0,0,0,0.7239
14994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,1,0,0.0,12.0,17.0,0,0,0.7152
14995,2150.0,215.0,2150.0,8681.0,215.0,0.0,0.0,215.0,0.0,215.0,...,0.0,1,1,1,0.0,329.0,2.0,3,1,0.5904
14996,0.0,0.0,0.0,51227.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,1,3,0.0,30.0,49.0,0,1,0.8525


In [ ]:
pred = []
for i in range(len(predictions['Score'])):
    if predictions['Label'][i] =='1.0':
        pred.append(predictions['Score'][i])
    else:
        pred.append(1-predictions['Score'][i])

In [ ]:
print(sample_submission.shape)

(14999, 2)


In [ ]:
set(sample_submission["user_id"]) - set(test_err["user_id"])

{43262}

In [ ]:
pred[:5]

[0.04920000000000002,
 0.24270000000000003,
 0.2449,
 0.19379999999999997,
 0.27490000000000003]

In [ ]:
id_pred = pd.DataFrame({"user_id":test_err["user_id"].unique()})

In [ ]:
id_pred.sort_values(by="user_id",inplace=True)

In [ ]:
id_pred["problem"] = pred

In [ ]:
id_pred

,user_id,problem
0,30000,0.0492
1,30001,0.2427
2,30002,0.2449
3,30003,0.1938
4,30004,0.2749
...,...,...
14993,44994,0.2761
14994,44995,0.2848
14995,44996,0.4096
14996,44997,0.1475


In [ ]:
sample_submission = pd.concat([id_pred,pd.DataFrame({"user_id":[43262],"problem":[0]})]).sort_values(by="user_id").reset_index(drop=True)
sample_submission

,user_id,problem
0,30000,0.0492
1,30001,0.2427
2,30002,0.2449
3,30003,0.1938
4,30004,0.2749
...,...,...
14994,44994,0.2761
14995,44995,0.2848
14996,44996,0.4096
14997,44997,0.1475


In [ ]:
sample_submission.to_csv("Jupyter Temp/lg 공모전/submissions/dacon_baseline_0120(1).csv", index = False)
sample_submission

,user_id,problem
0,30000,0.0492
1,30001,0.2427
2,30002,0.2449
3,30003,0.1938
4,30004,0.2749
...,...,...
14994,44994,0.2761
14995,44995,0.2848
14996,44996,0.4096
14997,44997,0.1475
